# Part 2

----------

### The provided datasets are integrated to link drug modalities with the subcellular locations of their targets, ensuring comprehensive data preparation.

Continuing on from the previous inspection of the 3 datasets downloaded, we now intend to integrate the datasets to effectively link drug modalities (types or classes of drugs) to the subcellular locations of their targets.

To do this we first link the id's present in each dataset: 
* DRUG --> MoA --> TARGET
* (id) --> [(chemblIds), (targetType)] --> (id)

Once this is established we can look specifically at patterns/distributions between drug modality and subcellular locations for each interaction.

In [1]:
# since the mechanism of action dataset has the fewest rows this will be the limiting factor when combining matching id's between all datasets.
# LOGIC: we want all merged mechanism of action dataset rows WHERE chemblIds == id (from merged drug info df) AND targets == id (from marged targets df)

In [6]:
# firstly columns need to be converted to same data types for comparisons, then merge based on the specific conditions highlighted above
import pandas as pd

merged_drug_info = pd.read_parquet("./data/merged_drug_info.parquet")
merged_mechanism = pd.read_parquet("./data/merged_mechanism.parquet")
merged_targets = pd.read_parquet("./data/merged_targets.parquet")

# STEP 1: need to convert columns we need to compare into the same data format and type
# convert 'chemblIds' and 'targets' columns to strings
merged_mechanism['chemblIds'] = merged_mechanism['chemblIds'].astype(str)
merged_mechanism['targets'] = merged_mechanism['targets'].astype(str)
# remove brackets and quotes from 'chemblIds' and 'targets' columns to we can compare them 
merged_mechanism['chemblIds'] = merged_mechanism['chemblIds'].str.strip("[]").str.replace("'", "")
merged_mechanism['targets'] = merged_mechanism['targets'].str.strip("[]").str.replace("'", "")

# STEP 2: merge on conditions
# merge drug info id and mechanism table chemblIds if both are the same 
merged_info_mechanism = pd.merge(merged_drug_info, merged_mechanism, left_on='id', right_on='chemblIds', how='inner') 
# do the same for identicle mechanism table targets and target table ids
merged_all = pd.merge(merged_info_mechanism, merged_targets, left_on='targets', right_on='id', how='inner')

# both the above steps create a dummy column each, id_x and id_y

# resulting dataframe dimension data
print("Shape of merged DataFrame:", merged_all.shape)
print("Columns of merged DataFrame:", merged_all.columns)

# save it to /data
merged_all.to_parquet('./data/MERGED_DATA_OUTPUT.parquet', index=False)


Shape of merged DataFrame: (3528, 53)
Columns of merged DataFrame: Index(['id_x', 'canonicalSmiles', 'inchiKey', 'drugType', 'blackBoxWarning',
       'name', 'yearOfFirstApproval', 'maximumClinicalTrialPhase', 'parentId',
       'hasBeenWithdrawn', 'isApproved', 'tradeNames', 'synonyms_x',
       'crossReferences', 'childChemblIds', 'linkedDiseases', 'linkedTargets',
       'description', 'actionType', 'mechanismOfAction', 'chemblIds',
       'targetName', 'targetType', 'targets', 'references', 'id_y',
       'approvedSymbol', 'biotype', 'transcriptIds', 'canonicalTranscript',
       'canonicalExons', 'genomicLocation', 'alternativeGenes', 'approvedName',
       'go', 'hallmarks', 'synonyms_y', 'symbolSynonyms', 'nameSynonyms',
       'functionDescriptions', 'subcellularLocations', 'targetClass',
       'obsoleteSymbols', 'obsoleteNames', 'constraint', 'tep', 'proteinIds',
       'dbXrefs', 'chemicalProbes', 'homologues', 'tractability',
       'safetyLiabilities', 'pathways'],
      

In [8]:
# here I want to quickly check if the merging process for the datasets has worked

import pandas as pd

merged_data = pd.read_parquet('./data/MERGED_DATA_OUTPUT.parquet')

# if drug_info id == chemblIds AND targets == targets id FOR all rows in the merged dataset THEN return True

condition_true_for_each_row = (merged_data['id_x'] == merged_data['chemblIds']) & (merged_data['targets'] == merged_data['id_y'])
all_conditions_true = condition_true_for_each_row.all()

# if output = true, we have successfully linked drug info to targets using the mechanism of action dataset as a bridging variable
print("Condition is true for each row:", all_conditions_true)

# visual confirmation of this also, sample a row
matching_rows = merged_data[(merged_data['id_x'] == merged_data['chemblIds']) & (merged_data['targets'] == merged_data['id_y'])]
random_matching_row = matching_rows.sample(1)
print("\n\nRandom matching row:")
print(random_matching_row[['id_x', 'chemblIds', 'targets', 'id_y']])

Condition is true for each row: True


Random matching row:
               id_x      chemblIds          targets             id_y
3056  CHEMBL2104753  CHEMBL2104753  ENSG00000132170  ENSG00000132170


-----------

#### At this point we have a single table with matching drugs, machanisms of action and their targets (alongside all of the other data within the rows)

#### Now we can begin analysis of specific type of drug (drug modality) and the subcellular locations of their targets

Columns of interest: 
* drug modalities                         ---> drugType
* subcellular locations of their targets  ---> subcellularLocations

-----------

In [9]:
# specifically extracting the columns of interest that we need to investigate the investigation at hand

import pandas as pd
from tabulate import tabulate

merged_data = pd.read_parquet('./data/MERGED_DATA_OUTPUT.parquet')

# obtain ONLY location value from each the dictionaries as part of the subcellularLocations data
merged_data['subcellularLocations'] = merged_data['subcellularLocations'].apply(lambda x: [item['location'] for item in x])

# create a new df with only 'drugType' and 'subcellularLocations' columns
final_df = merged_data[['drugType', 'subcellularLocations']]

final_df['subcellularLocations'] = final_df['subcellularLocations'].astype(str)

# save it in /data
final_df.to_parquet('./data/FINAL_DF.parquet', index=False)

# visualise to make sure it looks accurate
print(tabulate(final_df.head(10), headers='keys', tablefmt='grid'))

print(final_df.head(20))


+----+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | drugType       | subcellularLocations                                                                                                                                                                                          |
+====+================+===============================================================================================================================================================================================================+
|  0 | Small molecule | ['Cell membrane', 'Postsynaptic cell membrane', 'Basolateral cell membrane', 'Endoplasmic reticulum membrane', 'Plasma membrane']                                                                             |
+----+----------------+-------------------------------------------------

C:\Users\Yohen\AppData\Local\Temp\ipykernel_18424\322449198.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['subcellularLocations'] = final_df['subcellularLocations'].astype(str)


-------------------

#### Now we have a large table linking all of the various drug types with the subcellular locations whereby they have been studied to have some form of mechanism of action in.

#### In the next notebook, we will begin aggregating this data and conducting analysis on the distribution of drug modalities across various subcellular locations.

--------

### [Click here to go to PART 3 - exploratory data analysis stages](.\3_EDA.ipynb)